# Creating the webscraper- 04/04/2020

**Purpose:**
To create the script for pulling down the latest Covid-19 cases in Germany from the RKI website.

**Output:**
Code that can be modified for automatic daily scraping deployed on AWS.

In [1]:
from pathlib import Path
import os
from datetime import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import urllib

In [2]:
base_dir= Path.cwd().parents[0]
data_dir= base_dir/'data'
print(f'Base dir is: {base_dir}\ndata dir is: {data_dir}')

Base dir is: /home/darren/Documents/DS/projects/Covid-19_Germany
data dir is: /home/darren/Documents/DS/projects/Covid-19_Germany/data


In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
url=  'https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html'

req= urllib.request.Request(url, headers= headers)

with urllib.request.urlopen(req) as f:
    page= f.read()
    
soup= BeautifulSoup(page, 'html.parser')

In [4]:
"""
table= soup.find('table')
table_rows= table.find_all('tr')

for tr in table_rows:
    data= tr.find_all('td')
    row= [i.text for i in data]
    print(row)
"""

"\ntable= soup.find('table')\ntable_rows= table.find_all('tr')\n\nfor tr in table_rows:\n    data= tr.find_all('td')\n    row= [i.text for i in data]\n    print(row)\n"

In [77]:
dfs= pd.read_html(page, header=1, encoding='utf-8', thousands='.', decimal=',')

In [78]:
new_name= ['region', 'number', 'new', 'per 100k residents', 'deaths']
dfs[0].columns= new_name

In [79]:
new_df= dfs[0].drop(['number', 'per 100k residents', 'deaths'], axis=1)
new_df= new_df.drop(dfs[0].index[16], axis=0)
new_df

,region,new
0,Baden-Württem­berg,781.000
1,Bayern,1.128
2,Berlin,57.000
3,Brandenburg,40.000
4,Bremen,3.000
5,Hamburg,48.000
6,Hessen,99.000
7,Mecklenburg- Vor­pommern,5.000
8,Niedersachsen,248.000
9,Nordrhein-West­falen,649.000


In [80]:
cases= new_df['new']
cases_correct= [int(n) if (n%1 == 0) else int(n*1000) for n in num]

In [81]:
new_df[datetime.now()]= cases_correct

In [82]:
new_df= new_df.drop('new', axis=1)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   region                      16 non-null     object
 1   2020-04-06 19:31:39.110895  16 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 384.0+ bytes


In [84]:
new_df.to_csv(data_dir/'DE_new_cases.csv')